In [1]:
import chipwhisperer as cw
import sys
sys.path.append("../imports")
from helpers import *
from scheme import Scheme
from taskers import *

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()
scope.adc.samples=24400

In [ ]:
scope.dis()
target.dis()
exit()

In [9]:
# masking outside component
# function
def setup():
    print(start_time)
    # load(scope,"masked")
    # load(scope, "../c-files/masked-syndrome-CWLITEARM")
    load(scope, "masked-decode-inside", env)
    pass
    
def execute(index, op, t):
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    if op == "k0":
        p_temp = p0
        k_temp = k0
    elif op == "k1":
        p_temp = p1
        k_temp = k1
    elif op == "k0err":
        p_temp = p0_err.sample(byte=False)
        k_temp = k0
    elif op == "k1err":
        p_temp = p1_err.sample(byte=False)
        k_temp = k1
    else:
        raise "ERROR"

    rand = [random.randrange(2) for _ in range(64)]
    mem = xor(p_temp, s.encode(k_temp))
    posture(target,'k', get_bytes(mem), 16)
    return get_bytes(p_temp + [0] + s.encode(rand))

def ending(fdb, op, t):
    # runs after each CW command -> no return
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    pass
s = Scheme() # scheme execution helper

p0 = [random.randrange(2) for _ in range(127)]
p0_err = PUF(p0, 0.4)
k0 = [random.randrange(2) for _ in range(64)]
k1 = [random.randrange(2) for _ in range(64)]
p1 = xor(p0, s.encode(k1), s.encode(k0))
p1_err = PUF(p1, 0.4)


prefix_name = "MASKED"
    

env = ENV()
# operation settings
env.root = "D:/Programs/traces"
env.execute = execute
env.ending = ending
env.operations = operations = ["k0", "k1", "k0err", "k1err"] # operations list , "set2", "set3"
env.subfolder = "masked_hd_diff5" # subfolder name
env.TRACES_NUM = 800 # number of traces
scope.adc.samples = 24400 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 16
env.desc = ""
env.title = ""
env.name = prefix_name
env.readme = """masking done inside the component


PUF error model:
0.6 stable 10e-5 - 10e-6
0.4 unstable 10e-3 - 3*10e-2
"""


# run loop
start_time = date_str()
print(setup())
time.sleep(0.1)


file_names = [f"file{i}" for i in range(1)]
for file_name in file_names:
    status = traces(file_name, scope, target, env)
    

# 1
env.title = "Codeword masking, mask <= $ ; \{f_{k1}, f_{PUF}\} vs \{f_{k2}, f_{PUF}\}"
env.name = prefix_name + "_1"
env.desc = ""
split_file(env, file_names, [0, 1], "group1")

# 2
env.title = "Codeword masking, mask <= $ ; \{f_{k1}, f_{PUF}\} vs \{f_{k2}, f_{PUF}\}"
env.name = prefix_name + "_2"
env.desc = "PUF with error"
split_file(env, file_names, [2, 3], "group2")

remove_files(env, file_names)

env.name = prefix_name
generate_file(env, globals(), "start_time", "operations")
print("done", date_str())

2024-10-28 00-48
Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 12407 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 12407 bytes
None
file0 done 2024-10-28 00-53
